# Logical Operators

This notebook demonstrates use of the `ge` infix function for the comparison of two tensors. In this particular example, the probablity density `f` at two-dimensional location `(x, y)` is compared against a value `u` drawn uniformly from the unit interval `[0, 1)`. If `f(x, y) >= u`, or `f(x, y) ge u`, the point `(x, y)` [is accepted, otherwise rejected](#Accept-or-Reject-Samples). Infix operators `eq` (`==`), `ge` (`>=`), `le` (`<=`), `gt` (`>`), `lt` (`<`) are implemented for these containers/tensors:
```
RandomAccessible
RandomAccessibleInterval
RealRandomAccessible
RealRandomAccessibleRealInterval
```
and for scalars with any of these tensors.

## Dependencies and Imports

In [1]:
// set up dependencies
// use local maven repository; not yet deployed to remote maven repositories.
@file:Repository("*mavenLocal")
@file:Repository("https://maven.scijava.org/content/groups/public")
@file:Repository("https://jitpack.io")

// uncomment to search in your local maven repo
// requires installation into local maven repository (./gradlew build publishToMavenLocal)
@file:DependsOn("org.ntakt:ntakt:0.1.0-SNAPSHOT")

// uncomment to search in jitpack (TODO)
// @file:DependsOn("com.github.saalfeldlab:ntakt:<tbd>")

%use lets-plot

In [2]:
import kotlin.math.PI
import kotlin.math.pow
import kotlin.random.Random
import net.imglib2.RandomAccessibleInterval as RAI
import org.ntakt.*
import net.imglib2.type.numeric.real.DoubleType
import net.imglib2.view.Views

## Set up Data

In [3]:
val rng = Random(100)
val dims = longArrayOf(600, 400)
val mean = dims.map { it / 2.0 }.toDoubleArray()
val sigma = dims.map { it / 10.0 }.map{ it * it }.toDoubleArray()
val sigmaInverse = sigma.map { 1.0 / it }.toDoubleArray()
val sigmaDeterminant = sigma.map { it * it }.sum()
val twoPiPow = (2*PI).pow(2)
val normalizationFactor = (twoPiPow * sigmaDeterminant).pow(-0.5)
val exponent = ntakt.function(2, { 0.0.asType() }) { p, t -> 
    val dx = p.getDoublePosition(0) - mean[0];
    val dy = p.getDoublePosition(1) - mean[1];
    t.set(-0.5  * (dx * dx * sigmaInverse[0] + dy * dy * sigmaInverse[1]))
}
val gaussianInfinite = exponent.exp() * normalizationFactor
val gaussian = gaussianInfinite.rastered.interval(*dims)
val uniform = ntakt.doubles(*dims) { rng.nextDouble() }

### Visualize Conditional Distributions

In [4]:
val bellCurvesAtY = 
    Views.hyperSlice(gaussian, 1, 100L).flatIterable.map { it.realDouble } +
    Views.hyperSlice(gaussian, 1, 200L).flatIterable.map { it.realDouble } +
    Views.hyperSlice(gaussian, 1, 250L).flatIterable.map { it.realDouble }
val dY = mapOf<String, Any>(
    "X" to DoubleArray(dims[0].toInt()) { it.toDouble() }.let { it + it + it },
    "bell curve at y" to bellCurvesAtY,
    "y" to Array(dims[0].toInt()) { "100" } + Array(dims[0].toInt()) { "200" }  + Array(dims[0].toInt()) { "250" } 
)
val p = lets_plot(dY) { x = "X"; color = "y" }
p + 
    geom_line  { y = "bell curve at y" } +
    ggsize(800, 500)

In [5]:
val bellCurvesAtX = 
    Views.hyperSlice(gaussian, 0, 200L).flatIterable.map { it.realDouble } +
    Views.hyperSlice(gaussian, 0, 300L).flatIterable.map { it.realDouble } +
    Views.hyperSlice(gaussian, 0, 350L).flatIterable.map { it.realDouble }
val dX = mapOf<String, Any>(
    "X" to DoubleArray(dims[1].toInt()) { it.toDouble() }.let { it + it + it },
    "bell curve at x" to bellCurvesAtX,
    "x" to Array(dims[1].toInt()) { "200" } + Array(dims[1].toInt()) { "300" }  + Array(dims[1].toInt()) { "350" } 
)
val p = lets_plot(dX) { x = "X"; color = "x" }
p + 
    geom_line  { y = "bell curve at x" } +
    ggsize(800, 500)

## Accept or Reject Samples

In [6]:
val sampled = gaussian ge uniform * normalizationFactor
val points = sampled.where()

### Visualize samples

In [7]:
val scatterData = mapOf<String, Any>(
    "X" to points.map { it.getDoublePosition(0) } + listOf(mean[0]),
    "Y" to points.map { it.getDoublePosition(1) } + listOf(mean[1]),
    "label" to Array(points.size) { "sample" } + arrayOf("mean"),
    "alpha" to DoubleArray(points.size) { 0.1 } + doubleArrayOf(1.0)
)
lets_plot(scatterData) { x = "X"; y = "Y"; color = "label"; alpha = "alpha" } +
    geom_point(size = 3.0) +
    geom_density2d(color="black") +
    ggsize(900, 600)

In [8]:
val sampleMean = Pair(
    points.map{ it.getDoublePosition(0) }.sum() / points.size,
    points.map{ it.getDoublePosition(1) }.sum() / points.size
)
val (meanX, meanY) = sampleMean
sampleMean

(300.0063829787234, 200.42094414893617)

In [9]:
val sampleVariance = points.fold(DoubleArray(3)) { m, p ->
    val dX = p.getDoublePosition(0) - meanX
    val dY = p.getDoublePosition(1) - meanY
    m[0] += dX*dX
    m[1] += dX*dY
    m[2] += dY*dY
    m    
}.map { it / (points.size - 1) }
sampleVariance

[3605.538891364714, 25.387697942795615, 1590.124476533876]

In [10]:
sigma[0] to sigma[1]

(3600.0, 1600.0)